# **Addestramento di una rete neurale Faster R-CNN per la detection di luoghi ad alto valore culturale**

Pezzo da aggiungere a /usr/local/lib/python3.6/dist-packages/albumentations/augmentations/bbox_utils.py

```
    bbox=list(bbox)

    for i in range(4):
      if (bbox[i]<0) :
        bbox[i]=0
      elif (bbox[i]>1) :
        bbox[i]=1

    bbox=tuple(bbox)
```



## Librerie


Clonazione del repository *vision* di *pytorch* per utilizzare alcune funzioni di utilità per l'addestramento e valutazione del modello.

In [1]:
%%shell

git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0       # Check current version

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../
cp references/detection/coco_eval.py ../

Cloning into 'vision'...
remote: Enumerating objects: 33155, done.
remote: Counting objects: 100% (5698/5698), done.
remote: Compressing objects: 100% (1246/1246), done.
remote: Total 33155 (delta 4529), reused 5437 (delta 4338), pack-reused 27457
Receiving objects: 100% (33155/33155), 40.47 MiB | 23.23 MiB/s, done.
Resolving deltas: 100% (25145/25145), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be376084 version check against PyTorch's CUDA version


Installazione della libreria albumentations con specifica versione 0.4.6.


Questa libreria verrà utilizzata per eseguire operazioni di trasformazione di *data augmentation* sui dati. 

In [2]:
! pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 4.1 MB/s 
     |████████████████████████████████| 948 kB 7.0 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65173 sha256=7764673dbf46dd6859e78e4d161b718d7cbf9178dca6c24046117459d8186ccf
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


Librerie da importare:

In [3]:
# Core libraries
import os
import re
import glob
import numpy as np
import random
import xml.etree.ElementTree as ET

# PyTorch
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
import transforms as T                          # Module transform  (helper functions)
import utils                                    # Module utils      (helper functions)
from engine import train_one_epoch, evaluate    # Module engine     (helper functions)

# Data Augmentation
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# Opencv
import cv2 as cv
from google.colab.patches import cv2_imshow

## Dataset

Sono stati individuati **24 punti di interesse** suddivisi in altrettante directories. 



Ogni punto di interesse contiene, a sua volta, altre tre directory:
- *imgs*: contiene le immagini di dimensione 900 x 1300;
- *annotations*: contiene le annotazioni (coordinate delle bounding box) di ogni immagine catturata;
- *bounding_box*: contiene le immagini con le bouding box individuate.


Il dataset è consultabile al seguente [link](https://drive.google.com/drive/folders/1MenVMP6C9Vuo7JJ-iesAd1KfsemAvSLy?usp=sharing)

### Connessione a Google Drive

Connessione a google drive per scaricare il dataset.

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/')

path = "gdrive/MyDrive/DataSet/DatasetMonuments/"   # Replace with your path

Mounted at /content/gdrive/


### Caricamento del dataset

La funzione *get_bounding_boxes* permette di ottenere le coordinate di tutte le bouding boxes di uno specifico file .xml dato in input.

La funzione *format_labels* permette semplicemente di far 'matchare' le labels ottenute dal nome delle loro directories con quelle memorizzate nel file .xml (annotazione) di una specifica immagine.

In [26]:
def get_bounding_boxes(file_xml):
  r"""
  Get all bounding boxes of a specific file xml.
  :param file_xml: file from which to get bounding boxes
  """

  bounding_box_list = []
  labels = []

  tree = ET.parse(file_xml)
  root = tree.getroot()

  # Get all bounding boxes
  j = 6
  while True:
      try:
          object = root[j]

          labels.append(object[0].text)

          box = object[4]
          x_min, y_min, x_max, y_max = int(box[0].text), int(box[1].text), int(box[2].text), int(box[3].text)
          bounding_box_list.append([[x_min, y_min], [x_max, y_max]])

          j += 1
          
      except:
          break

  return bounding_box_list, format_labels(labels)

def format_labels(list_labels):
  r"""
  Match the labels.
  :param list_label: list containing the labels to math.
  """

  new_list = []
  for label in list_labels:

    if label == "Centro Arti Visive Pescheria":
      new_list.append("Centro Arti Visive")

    elif label == "Giuseppe Garibaldi":
      new_list.append("Statua Giuseppe Garibaldi")

    elif label == "Scultura della Memoria":
      new_list.append("Scultura Della Memoria")

    elif label == "Giulio Perticari":
      new_list.append("Statua Giulio Perticari")

    elif label == "Gioachino Rossini":
      new_list.append("Statua Gioachino Rossini")
      
    elif label == "Palla di Pomodoro":
      new_list.append("Palla Di Pomodoro")

    elif label == "Arco di Trionfo":
      new_list.append("Arco Di Trionfo")

    elif label == "Palazzo delle Poste":
      new_list.append("Palazzo Delle Poste")

    elif label == "Fontana Piazza del Popolo":
      new_list.append("Fontana Piazza")
      
    else:
      new_list.append(label)
  
  return new_list

La funzione *get_transform* applica trasformazioni di *data augmentation* prima di darli in input alla rete.

In [6]:
def get_transform(train):
  r"""
  Apply the transformations.
  :param: true if it is need to apply the horizontal flip, false otherwise.
  """

  if train:
      return A.Compose([A.HorizontalFlip(0.5), ToTensorV2(p=1.0)],
                       bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
  else:
      return A.Compose([ToTensorV2(p=1.0)],
                       bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

Per ogni etichetta, verrà creata un'istanza della classe *DatasetCulturalHeritage*

In [49]:
class DatasetCulturalHeritage(Dataset):
  r"""
  Class to load the dataset of a specific label.
  """

  def __init__(self, label, label_ds, path, transforms=None, height=224, width=224):
      r"""
      Load the dataset.
      :param label: label of the dataset.
      :param label_ds: label of the dataset not formatted. 
      :param dir: directory of the monumnet/place.
      :param transform: transform to apply to the data.
      """

      self.label_name   = label
      self.label_ds     = label_ds
      self.img_list     = glob.glob(path + self.label_ds + '/imgs/*.JPG') + glob.glob(path + self.label_ds + '/imgs/*.jpg')
      self.annotation_list = glob.glob(path + self.label_ds + '/annotations/*.xml')

      self.img_list.sort()
      self.annotation_list.sort()

      self.transforms = transforms
      self.height = height
      self.width = width

      print(f"Classe [{self.label_name}] caricata!\n")
    
  def __getitem__(self, index):
      r"""
      Get the image and its bouding boxes.
      :param index: index of the specific image
      """
      global total_labels

      img_path, annotations_path = self.img_list[index], self.annotation_list[index]

      img = cv.imread(img_path)

      height, width, _ = img.shape

      img = cv.resize(img, (self.width, self.height), cv.INTER_AREA)
      img = cv.cvtColor(img, cv.COLOR_BGR2RGB).astype(np.float32)
      img /= 255.0

      # Bouding boxes
      bounding_boxes = []
      labels = []
      _boxes, _labels = get_bounding_boxes(annotations_path)

      for box, label in zip(_boxes, _labels):

        labels.append(total_labels.index(label))
      
        # Extracting scaled coordinates for the size of the current image
        x_min = (box[0][0] / width) * self.width
        x_max = (box[1][0] / width) * self.width
        y_min = (box[0][1] / height)* self.height
        y_max = (box[1][1] / height)* self.height

        bounding_boxes.append([x_min, y_min, x_max, y_max])

      if len(bounding_boxes) == 0:
        # Image without bounding box
        bb_tensor = torch.as_tensor([[0, 0, 0, 0]], dtype=torch.float32)

      else:
        # Image with bounding box
        bb_tensor = torch.as_tensor(bounding_boxes, dtype=torch.float32)

      # Labels
      labels = torch.as_tensor(labels, dtype=torch.int64)

      # Area
      area = (bb_tensor[:, 3] - bb_tensor[:, 1]) * \
              (bb_tensor[:, 2] - bb_tensor[:, 0])

      # Id
      img_id = torch.tensor([index])

      is_crowd = torch.zeros((len(bb_tensor),), dtype=torch.int64)

      target = {}
      target["boxes"] = bb_tensor
      target["labels"] = labels
      target["image_id"] = img_id
      target["area"] = area
      target["is_crowd"] = is_crowd

      if self.transforms is not None and len(bounding_boxes) > 0:

        result_transform = self.transforms(image = img,
                                           bboxes = target['boxes'],
                                           labels = labels)
        
        img = result_transform['image']
        target['boxes'] = torch.Tensor(result_transform['bboxes'])

      return img, target

  def length(self):
      return len(self.img_list), len(self.annotation_list)

  def __len__(self):
      assert len(self.img_list) == len(self.annotation_list)

      return len(self.img_list)

La funzione *split_directory* permette di formattare il testo dato in input sulla base delle lettere maiuscole.

In [8]:
def split_directory(title):
    r"""
    Split text of the directory
    :param title: text to split
    :return: text with the tabulations
    """
    
    title = title[0].upper() + title[1:]
    folder = re.findall('[A-Z][^A-Z]*', title)
    result = ""

    for i, item in enumerate(folder):
        result += str(item) + (" " if (i + 1) != len(folder) else "")

    return result

Caricamento del dataset.

In [54]:
global total_labels

dataset = []
num_imgs = 0
num_annotations = 0
total_labels = []

print("Caricamento dataset in corso ...\n\n")

dirs = os.listdir(path)

for index, dir in enumerate(dirs):

  name_class = split_directory(dir)

  class_label = DatasetCulturalHeritage(label=name_class, label_ds=dir, path=path, transforms=get_transform(train=True))

  _num_imgs, _num_annotations = class_label.length()
  num_imgs += _num_imgs
  num_annotations += _num_annotations

  total_labels.append(name_class)
  dataset.append(class_label)

print("Caricamento terminato.")

Caricamento dataset in corso ...


Classe [Villino Ruggeri] caricata!

Classe [Statua Gioachino Rossini] caricata!

Classe [Scultura Della Memoria] caricata!

Classe [Rocca Costanza] caricata!

Classe [Portale San Domenico] caricata!

Classe [Teatro Rossini] caricata!

Classe [Statua Giulio Perticari] caricata!

Classe [Statua Giuseppe Garibaldi] caricata!

Classe [Teatro Sperimentale] caricata!

Classe [Villa Caprile] caricata!

Classe [Palla Di Pomodoro] caricata!

Classe [Palazzo Ducale] caricata!

Classe [Parrocchia Santa Maria] caricata!

Classe [Palazzo Comunale] caricata!

Classe [Palazzo Delle Poste] caricata!

Classe [Palazzo Olivieri] caricata!

Classe [Chiesa San Agostino] caricata!

Classe [Fontana Piazza] caricata!

Classe [Centro Arti Visive] caricata!

Classe [Palazzo Baviera] caricata!

Classe [Arco Di Trionfo] caricata!

Classe [Casa Rossini] caricata!

Classe [Berlina Mosca] caricata!

Classe [Cattedrale Santa Maria Assunta] caricata!

Classe [Free] caricata!

Caricam

In [55]:
print(f"Numero totale di immagini: {num_imgs}\n")
print(f"Numero totale di annotazioni: {num_annotations}", end="\n\n")
print(f"Numero totale di classi: {len(total_labels)}")

Numero totale di immagini: 953

Numero totale di annotazioni: 953

Numero totale di classi: 25


### Visualizzazione del dataset

La funzione *stack_images* permette di mostrare una lista orizzontale e/o verticale di immagini.

In [11]:
def stack_images(scale, img_array):
    r"""
    Stack the images based on the number of them by rows and columns.
    Resize the images.
    :param scale: scale factor.
    :param img_array: array of images.
    :return: array of images to show.
    """

    rows = len(img_array)
    cols = len(img_array[0])

    rowsAvailable = isinstance(img_array[0], list)

    width = img_array[0][0].shape[1]
    height = img_array[0][0].shape[0]

    if rowsAvailable:
        for x in range(0, rows):
            for y in range(0, cols):

                if img_array[x][y].shape[:2] == img_array[0][0].shape[:2]:
                    img_array[x][y] = cv.resize(img_array[x][y], (0, 0), None, scale, scale)
                else:
                    img_array[x][y] = cv.resize(img_array[x][y], (img_array[0][0].shape[1], img_array[0][0].shape[0]),
                                                None, scale, scale)

                if len(img_array[x][y].shape) == 2:
                    img_array[x][y] = cv.cvtColor(img_array[x][y], cv.COLOR_GRAY2BGR)

        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank] * rows

        for x in range(0, rows):
            hor[x] = np.hstack(img_array[x])
        ver = np.vstack(hor)

    else:
        for x in range(0, rows):

            if img_array[x].shape[:2] == img_array[0].shape[:2]:
                img_array[x] = cv.resize(img_array[x], (0, 0), None, scale, scale)
            else:
                img_array[x] = cv.resize(img_array[x], (img_array[0].shape[1], img_array[0].shape[0]), None, scale,
                                         scale)

            if len(img_array[x].shape) == 2:
                img_array[x] = cv.cvtColor(img_array[x], cv.COLOR_GRAY2BGR)

        hor = np.hstack(img_array)
        ver = hor

    return ver


La funzione *draw_description* disegna il nome della classe in basso nell'immagine.

In [12]:
def draw_description(img, text):
    r"""
    Draw description image.
    :param text: image description
    :param img: image in which to insert the descrition
    :return: image with description
    """

    bottom = int(0.04 * img.shape[0])
    img = cv.copyMakeBorder(img, 0, bottom, 0, 0, cv.BORDER_CONSTANT, None, (255, 255, 255))

    height, _, _ = img.shape
    cv.putText(img, text, (0, height - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    
    return img

La funzione *convert_point_to_tuple* converte una coordinata (punto x,y) in una tupla.

In [13]:
def convert_point_to_tuple(point):
    x, y = point
    return (x,y)

Il codice seguente mostra una griglia di 4 immagini casuali prese dal dataset. Per ogni immagine, a sinistra, viene mostrata quella originale e a destra la stessa immagine con le bounding box individuate.

In [ ]:
labels_to_show = []       # Labels of the images to show
bounding_box_list = []    # List of the boudning boxes drawed.
imgs_read_list = []       # List of the images read (no trasformations)
imgs_list = []            # List of the images trasformated (with bouding boxes)
title_images_list = []    # Label formattated.

labels_to_show.append(random.randint(0, len(dirs) - 1))
labels_to_show.append(random.randint(0, len(dirs) - 1))
labels_to_show.append(random.randint(0, len(dirs) - 1))
labels_to_show.append(random.randint(0, len(dirs) - 1))

for i in range(0, 4):
    alpha = 0.5
    bounding_box_list.clear()

    index_label = labels_to_show[i]
    class_label = dataset[index_label]

    imgs = class_label.img_list
    annotations = class_label.annotation_list
    label = class_label.label_ds

    index_img = random.randint(0, len(imgs) - 1)

    # Parse xml
    tree = ET.parse(annotations[index_img])
    root = tree.getroot()

    # Get bounding boxes
    bounding_box_list, _ = get_bounding_boxes(annotations[index_img])

    # Image
    img_read = cv.imread(imgs[index_img])

    img_original = cv.resize(img_read, (400,400))
    imgs_read_list.append(img_original)

    # Mask
    mask = np.zeros_like(img_read[:, :, 0])

    for j, box in enumerate(bounding_box_list):
        # Draw rectangle
        cv.rectangle(img_read, convert_point_to_tuple(box[0]), convert_point_to_tuple(box[1]), (0, 0, 255), 5)

        polygon = np.array([[bounding_box_list[j][0]], [[bounding_box_list[j][1][0],
                            bounding_box_list[j][0][1]]], [bounding_box_list[j][1]],
                            [[bounding_box_list[j][0][0], bounding_box_list[j][1][1]]]])
        
        cv.fillConvexPoly(mask, polygon, 1)

    # Get polygon
    img = cv.bitwise_and(img_read, img_read, mask=mask)
    img = cv.addWeighted(img_read.copy(), alpha, img, 1 - alpha, 0)
    img = cv.resize(img, (400, 400))
    imgs_list.append(img)

    title_images_list.append(class_label.label_name)


imgs_stack = stack_images(1, ([draw_description(imgs_read_list[0], title_images_list[0]), draw_description(imgs_list[0], title_images_list[0])],
                              [draw_description(imgs_read_list[1], title_images_list[1]), draw_description(imgs_list[1], title_images_list[1])],
                              [draw_description(imgs_read_list[2], title_images_list[2]), draw_description(imgs_list[2], title_images_list[2])],
                              [draw_description(imgs_read_list[3], title_images_list[3]), draw_description(imgs_list[3], title_images_list[3])]))

cv2_imshow(imgs_stack)

### Trainining, validation e test set

La funzione *split_dataset* suddivide il dataset in training, validation e test set rispettivamente al 70% (training), 20% (validation) e 10% (test).

In [15]:
def split_dataset(dataset):
    r"""
    Slip dataset in training, validation and test set:
        - 70% training set;
        - 20% validation set:
        - 10% test set.

    :param dataset: dataset to split
    :return : training, validation and test set.
    """
    length_dataset = len(dataset)

    length_train = np.int_(length_dataset * 0.7)
    length_validate = np.int_(length_dataset * 0.2)

    training_dataset = Subset(dataset, range(0, length_train))
    validation_dataset = Subset(dataset, range(length_train, length_train + length_validate))

    dataset.transofrms = get_transform(train=False)     # Disabled the transforms for test set
    test_dataset = Subset(dataset, range(length_train + length_validate, len(dataset)))

    return training_dataset, validation_dataset, test_dataset

Partizionamento dataset.

In [56]:
trainining_set = []
validation_set = []
test_set = []

for _set in dataset:
    
    training_dataset, validation_dataset, test_dataset = split_dataset(_set)

    trainining_set.append(training_dataset)
    validation_set.append(validation_dataset)
    test_set.append(test_dataset)

training_dataset, validation_dataset, test_dataset = ConcatDataset(trainining_set), ConcatDataset(validation_set), \
                                                     ConcatDataset(test_set)

print(f"** Dataset caricato correttamente! Totale immagini: {len(training_dataset) + len(validation_dataset) + len(test_dataset)} **\n")
print(f"Dimensione training: {len(training_dataset)} immagini - (70%)")
print(f"Dimensione validation: {len(validation_dataset)} immagini - (20%)")
print(f"Dimensione test: {len(test_dataset)} immagini - (10%)\n\n")
print(f"Totale immagini partizionate: {len(training_dataset) + len(validation_dataset) + len(test_dataset)}\n\n")

** Dataset caricato correttamente! Totale immagini: 953 **

Dimensione training: 657 immagini - (70%)
Dimensione validation: 181 immagini - (20%)
Dimensione test: 115 immagini - (10%)


Totale immagini partizionate: 953




Creazione del *DataLoader* rispettivamente per training, validation e test set.

In [57]:
batch_size = 8

# Training set
training_loader = DataLoader(training_dataset,
                             batch_size=batch_size,
                             shuffle=True,
                             num_workers=4,
                             collate_fn=utils.collate_fn)

# Validation set
validation_loader = DataLoader(validation_dataset,
                               batch_size=batch_size,
                               shuffle=True,
                               num_workers=4,
                               collate_fn=utils.collate_fn)

# Test set
test_loader = DataLoader(test_dataset,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=4,
                         collate_fn=utils.collate_fn)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Modello della rete

La funzione *get_model* inizializza la rete neurale Faster R-CNN preaddestrata.

In [18]:
def get_model(num_classes):
  r"""
  Model initialization.
  :param num_classes: number of classes of the problem.
  """

  # Load the model pretrained
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

  # Get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  return model

Modifica del modello aggiungendo una specifica backbone.

In [ ]:
# backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# backbone.out_channels = 1280

# anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
#                                   aspect_ratios=((0.5, 1.0, 2.0),))

# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
#                                                output_size=7,
#                                                sampling_ratio=2)


Definizione del modello

In [19]:
# Classes
num_classes = len(total_labels) + 1  # N classes + background

# Model
model = get_model(num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Runtime used: {device}\n\n")
model = model.to(device)

params = [p for p in model.parameters() if p.requires_grad]

# Optimizer
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9,
                            weight_decay=0.0005)

# learning rate withStep LR
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# Epoch
num_epochs = 5

print("** SUMMARY TRAINING **\n")
print(f"- Classes of the problem: {num_classes};")
print(f"- Epochs number: {num_epochs};")
print(F"- Optimizer: {optimizer.__class__.__name__},")
print(f"             Learning Rate: {optimizer.defaults['lr']},")
print(f"             Momentum: {optimizer.defaults['momentum']}.")

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth



Runtime used: cuda


** SUMMARY TRAINING **

- Classes of the problem: 25;
- Epochs number: 5;
- Optimizer: SGD,
             Learning Rate: 0.005,
             Momentum: 0.9.


## Addestramento

In [58]:
for epoch in range(num_epochs):
    
    # Training 
    train_one_epoch(model, optimizer, training_loader, device, epoch, print_freq=10)

    # Update LR
    lr_scheduler.step()

    # Evaluation
    evaluate(model, test_loader, device=device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TypeError: ignored

## Risultati ottenuti

Note:

- Verificare se per il test set, le operazioni di data augmentation non vengono effettuate;

- Valutare se aggiungere la modifica della backbone della rete;


Errori:

- bug di albumentations con i riquadri delle bb;

- non riesco a creare bounding box per immagini senza oggetti (label free)

- capire come allenare i 3 ds: traininf, validation e test (è possibile usare più volte la funzione di pytorch ??)